#### Семинарская задача 2 (2 балла).

Получать оценки методом рандома - грустно. Попробуйте придумать что-нибудь, чтобы оценки за домашку зависели от времени, которое на нее потратил студент (но при этом неплохо бы поставить какое-нибудь ограничение, чтобы студент не мог сидеть над домашкой 100500 часов!), а оценки за ту часть экзамена, которая сдается непосредственно на экзамене - от ранее полученных оценок (ну типа, если человек весь семестр учился хорошо и получал одни пятерки, шанс, что он на самом экзамене ответит на 0 баллов, крайне невелик...)

In [ ]:
import random, time
from collections import defaultdict

class Student:
  def __init__(self, name: str, course: int):
    self.name = name
    self.course = course
    self.recordbook = RecordBook()

  def sleep(self, n: int):
    print(f'{self.name} спит')
    time.sleep(n)
    print(f'{self.name} проснулся')

  def attend(self, lecture, n):
    print(f'{self.name} приходит на пару по: {lecture.discipline}')
    time.sleep(n)

  def do_homework(self, lecture, task_numb): # добавлю количество задач в дз для подсчета времени и баллов
    print(f'{self.name} делает домашку по: {lecture.discipline}')

    my_time = task_numb * lecture.diffic_lev // 10

    print(f'Делать домашку придется: {my_time} часов...')

    for i in range(my_time):
      time.sleep(1)
      print('Скучно' + '.' * random.randint(1, 10))

    if my_time > 30: # зависимость оценки за дз от потраченного времени и кол-ва задач
      print('слишком долго делаем домашку!')
    elif 1 <= my_time <= 10:
      my_mark = task_numb * 3
    elif 11 <= my_time <= 20:
      my_mark = task_numb * 2
    else:
      my_mark = task_numb

    self.recordbook.addmark(lecture.discipline, my_mark)

  def passexam(self, lecture):
    if self.recordbook.marks[lecture] >= 50: # будем считать, что человек, набравший хотя бы 50 баллов за семестр, на экзамене не наберет меньше 20, и наоборот
      mark = random.randint(20, 40)
      self.recordbook.addmark(lecture, mark)
    else:
      mark = random.randint(0, 19)
      self.recordbook.addmark(lecture, mark)

    print(f'{self.name} получает за экзамен: {mark}')
    if self.recordbook.marks[lecture] < 50:
      print(f'{self.name} идет на пересдачу!')
    else:
      print('Ура! Экзамен сдан!')

class RecordBook:
  def __init__(self):
    self.marks = defaultdict(int)

  def addmark(self, discipline, mark):
    self.marks[discipline] += mark

  def showmarks(self):
    for d in self.marks:
      print(f'Дисциплина "{(d)}": {self.marks[d]}')

class Discipline:
  def __init__(self, discipline, needed_time, diffic_lev):
    self.discipline = discipline
    self.time = needed_time
    self.diffic_lev = diffic_lev

In [ ]:
prog = Discipline('Программирование', 2, 12)

vasya = Student('Вася', 2)
vasya.attend(prog, 3)
vasya.do_homework(prog, 3)
vasya.do_homework(prog, 4)
vasya.recordbook.showmarks()
vasya.passexam('Программирование')
vasya.recordbook.showmarks()

# с таким инпутом Вася должен получить за семинары 21 балл

Вася приходит на пару по: Программирование
Вася делает домашку по: Программирование
Делать домашку придется: 3 часов...
Скучно..
Скучно..........
Скучно...
Вася делает домашку по: Программирование
Делать домашку придется: 4 часов...
Скучно.........
Скучно..
Скучно....
Скучно......
Дисциплина "Программирование": 21
Вася получает за экзамен: 3
Вася идет на пересдачу!
Дисциплина "Программирование": 24


#### Задача 1 (5 баллов).

Перепишите задачи 5-8 из первого семинара в класс. У класса должен быть главный метод "считать конлу" и дополнительные методы для подсчета. Неплохо также написать отдельный класс Token для токенов - каждое слово или знак пунктуации в формате описывается 10 значениями, разделенными знаком табуляции, подробнее можно почитать [тут](https://universaldependencies.org/format.html).

In [ ]:
!wget https://raw.githubusercontent.com/rsuh-python/bac21-25/main/term06/01-Exercise/syntagrus_tiny.conllu

--2024-03-02 20:29:30--  https://raw.githubusercontent.com/rsuh-python/bac21-25/main/term06/01-Exercise/syntagrus_tiny.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6659 (6.5K) [text/plain]
Saving to: ‘syntagrus_tiny.conllu’

syntagrus_tiny.conl 100%[===================>]   6.50K  --.-KB/s    in 0s      

2024-03-02 20:29:31 (53.3 MB/s) - ‘syntagrus_tiny.conllu’ saved [6659/6659]



In [ ]:
from string import punctuation
from collections import Counter

class Conllu:
  def __init__(self, conllu: str):
    self.conllu = conllu
    self.lines = []

  def read_conllu(self):
    result = {}

    with open(self.conllu, 'r', encoding='utf-8') as file:
      self.lines = file.readlines()

    for line in self.lines:
      if line.startswith('# sent_id'):
        id = line.strip().split('= ')[1]
        result[id] = {'text': '', 'tokens': []}

      elif line.startswith('# text'):
        text = line.strip().split('= ')[1]
        result[id]['text'] = text

      elif line[0].isdigit():
        word_str = line.strip().split("\t")
        word = word_str[1].strip(punctuation)
        if word != '': # это чтобы точки после strip(punctuation) не вошли как пустая строка в словарь
          result[id]['tokens'].append(word)

    return result

  def poscount(self):
    count = []

    for line in self.lines:
      if line[0].isdigit():
        pos = line.strip().split()[3]
        count.append(pos)

    count = Counter(count)

    print(f'POS{"":>13}%')

    word_sum = 0
    for value in count.values():
      word_sum += value

    for key, value in count.items():
      indent = 15 - len(key) - len(str(round((value/word_sum * 100))))
      print(f'{key}' + " " * indent + f'{round((value/word_sum * 100), 1)}')

  def unique_words(self):
    unique_words = []

    for line in self.lines:
      if line[0].isdigit():
        my_line = line.strip().split()
        if my_line[3] != 'PUNCT' and my_line[2] not in unique_words:
          unique_words.append(my_line[2])

    print(unique_words)
    print(f'Количество уникальных слов: - {len(unique_words)}.')

  def lemmas(self):
    a = 0

    for line in self.lines:
      if line[0].isdigit():
        my_line = line.strip().split()
        if my_line[1] == my_line[2]:
          a += 1

    print(f'Количество слов, которые по форме совпадают со своей леммой: {a}.')

In [ ]:
my_conllu = Conllu('/content/syntagrus_tiny.conllu')
my_conllu.read_conllu()

{'2003Armeniya.xml_1': {'text': 'В советский период времени число ИТ- специалистов в Армении составляло около десяти тысяч.',
  'tokens': ['В',
   'советский',
   'период',
   'времени',
   'число',
   'ИТ',
   'специалистов',
   'в',
   'Армении',
   'составляло',
   'около',
   'десяти',
   'тысяч']},
 '2003Armeniya.xml_2': {'text': 'Доставшийся в наследство от советского периода времени промышленный и интеллектуальный потенциал оказался благом и горем страны.',
  'tokens': ['Доставшийся',
   'в',
   'наследство',
   'от',
   'советского',
   'периода',
   'времени',
   'промышленный',
   'и',
   'интеллектуальный',
   'потенциал',
   'оказался',
   'благом',
   'и',
   'горем',
   'страны']},
 '2003Armeniya.xml_3': {'text': 'С одной стороны, квалифицированные кадры и развитая производственная инфраструктура резко отличали Армению от других регионов СССР, где доминировали добывающие отрасли, а экономика строилась на поставке сырьевых ресурсов.',
  'tokens': ['С',
   'одной',
   'стор

In [ ]:
my_conllu.poscount()
my_conllu.unique_words()
my_conllu.lemmas()

POS             %
ADP          12.7
ADJ          11.1
NOUN         31.7
PROPN         6.3
PUNCT        11.1
VERB         14.3
NUM           3.2
CCONJ         6.3
ADV           3.2
['в', 'советский', 'период', 'время', 'число', 'ИТ', 'специалист', 'Армения', 'составлять', 'около', 'десять', 'тысяча', 'достаться', 'наследство', 'от', 'промышленный', 'и', 'интеллектуальный', 'потенциал', 'оказаться', 'благо', 'горе', 'страна', 'с', 'один', 'сторона', 'квалифицировать', 'кадр', 'развить', 'производственный', 'инфраструктура', 'резко', 'отличать', 'другой', 'регион', 'СССР', 'где', 'доминировать', 'добывать', 'отрасль', 'а', 'экономика', 'строить', 'на', 'поставка', 'сырьевой', 'ресурс']
Количество уникальных слов: - 47.
Количество слов, которые по форме совпадают со своей леммой: 30.


In [ ]:
class Token:
  def __init__(self, conllu: str):
    self.conllu = conllu
    self.lines = []

  def read_tokens(self):

    with open(self.conllu, 'r', encoding='utf-8') as file:
      self.lines = file.readlines()

    for line in self.lines:
      if line[0].isdigit():
        word_str = line.split("\t")
        all_tokens = ['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC']
        token_result = dict(zip(all_tokens, word_str))
        print(token_result)

In [ ]:
my_conllu = Token('/content/syntagrus_tiny.conllu')
my_conllu.read_tokens()

{'ID': '1', 'FORM': 'В', 'LEMMA': 'в', 'UPOS': 'ADP', 'XPOS': '_', 'FEATS': '_', 'HEAD': '3', 'DEPREL': 'case', 'DEPS': '3:case', 'MISC': '_\n'}
{'ID': '2', 'FORM': 'советский', 'LEMMA': 'советский', 'UPOS': 'ADJ', 'XPOS': '_', 'FEATS': 'Animacy=Inan|Case=Acc|Degree=Pos|Gender=Masc|Number=Sing', 'HEAD': '3', 'DEPREL': 'amod', 'DEPS': '3:amod', 'MISC': '_\n'}
{'ID': '3', 'FORM': 'период', 'LEMMA': 'период', 'UPOS': 'NOUN', 'XPOS': '_', 'FEATS': 'Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing', 'HEAD': '11', 'DEPREL': 'obl', 'DEPS': '11:obl:в:acc', 'MISC': '_\n'}
{'ID': '4', 'FORM': 'времени', 'LEMMA': 'время', 'UPOS': 'NOUN', 'XPOS': '_', 'FEATS': 'Animacy=Inan|Case=Gen|Gender=Neut|Number=Sing', 'HEAD': '3', 'DEPREL': 'nmod', 'DEPS': '3:nmod:gen', 'MISC': '_\n'}
{'ID': '5', 'FORM': 'число', 'LEMMA': 'число', 'UPOS': 'NOUN', 'XPOS': '_', 'FEATS': 'Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing', 'HEAD': '11', 'DEPREL': 'obj', 'DEPS': '11:obj', 'MISC': '_\n'}
{'ID': '6', 'FORM': 'ИТ', 'LEMM

#### Задача 2 (5 баллов).

Хотим написать программу - базу данных для библиотеки. Нам понадобится класс "книга", который будет содержать автора, название, жанр и количество страниц; а также класс "библиотека", в котором в атрибутах будут сидеть все наши книги. В библиотеку будут поступать запросы: нужно будет выдавать перечень всех книг одного автора, перечень всех книг конкретного жанра, а также проверять, что книга такого автора и с таким названием есть (вам хорошо помогут генераторные выражения для всех этих вещей, а еще можно переопределить магический метод \_\_eq\_\_ у класса "книга" - он должен возвращать bool, а внутри него можно сравнивать атрибуты экземпляров self и other - и проверять наличие объекта класса "книга" в списке).

In [81]:
class Book:
  def __init__(self, author, title, genre, pages: int):
    self.author = author
    self.title = title
    self.genre = genre
    self.pages = pages

  def __str__(self):
    return f'Автор: {self.author}, название: "{self.title}", жанр: {self.genre}, кол-во страниц: {self.pages}'

class Library:
  def __init__(self):
    self.all_books = []

  def add_book(self, book): # добавить новую книгу в библиотеку
    self.all_books.append(book)

  def show_library(self): # показать всю библиотеку
    for i in self.all_books:
      print(i)

  def all_books_by_author(self, author): # перечень всех книг одного автора
    return [book for book in self.all_books if book.author == author]

  def all_books_by_genre(self, genre): #  перечень всех книг конкретного жанра
    return [book for book in self.all_books if book.genre == genre]

  def check_book(self, author, title): # проверяем, что книга такого автора и с таким названием есть
    return any(book for book in self.all_books if book == Book(author, title))

In [64]:
library = Library()
book_1 = Book('И. С. Тургенев', 'Ася', 'повесть', '64')
book_2 = Book('Антуан де Сент-Экзюпери', 'Военный лётчик', 'биография', '366')
book_3 = Book('И. С. Тургенев', 'Отцы и Дети', 'роман', '288')
book_4 = Book('Л. Н. Толстой', 'Война и Мир', 'роман', '1300')
library.add_book(book_1), library.add_book(book_2), library.add_book(book_3), library.add_book(book_4)

(None, None, None, None)

In [65]:
library.show_library()

Автор: И. С. Тургенев, название: "Ася", жанр: повесть, кол-во страниц: 64
Автор: Антуан де Сент-Экзюпери, название: "Военный лётчик", жанр: биография, кол-во страниц: 366
Автор: И. С. Тургенев, название: "Отцы и Дети", жанр: роман, кол-во страниц: 288
Автор: Л. Н. Толстой, название: "Война и Мир", жанр: роман, кол-во страниц: 1300


In [82]:
print(library.check_book('Л. Н. Толстой', 'Отцы и Дети')), print(library.check_book('Л. Н. Толстой', 'Война и Мир'))

False
True


(None, None)

In [66]:
library.all_books_by_author('И. С. Тургенев') # когда я добавила __str__, в show_library все стало красиво, а здесь нет..

[<__main__.Book at 0x781804725cf0>, <__main__.Book at 0x781804726590>]